In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, auc, roc_curve, classification_report, confusion_matrix, precision_score, recall_score, f1_score

In [86]:
df1 = pd.read_csv('./data/train.csv')
df2 = pd.read_csv('./data/train_dataset.csv')

df1.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


In [87]:
df1 = df1.drop(['id'], axis=1)


In [88]:
df = pd.concat([df1, df2])
df.shape

(198240, 23)

In [89]:
df.isnull().sum()


age                    0
height(cm)             0
weight(kg)             0
waist(cm)              0
eyesight(left)         0
eyesight(right)        0
hearing(left)          0
hearing(right)         0
systolic               0
relaxation             0
fasting blood sugar    0
Cholesterol            0
triglyceride           0
HDL                    0
LDL                    0
hemoglobin             0
Urine protein          0
serum creatinine       0
AST                    0
ALT                    0
Gtp                    0
dental caries          0
smoking                0
dtype: int64

In [90]:
df['bmi'] = df['weight(kg)'] / ((df['height(cm)'] / 100) ** 2)
df['waist_to_height_ratio'] = df['waist(cm)'] / df['height(cm)']
df['bp_index'] = df['systolic'] / df['relaxation']

In [91]:
from sklearn.preprocessing import MinMaxScaler
scaler = ['age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)',
       'eyesight(right)', 'hearing(left)', 'hearing(right)', 'systolic',
       'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride',
       'HDL', 'LDL', 'hemoglobin', 'Urine protein', 'serum creatinine', 'AST',
       'ALT', 'Gtp', 'dental caries', 'bmi', 'waist_to_height_ratio', 'bp_index']

mm = MinMaxScaler()

mm.fit(df[scaler])

df[scaler] = mm.transform(df[scaler])

df = pd.get_dummies(df)

df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking,bmi,waist_to_height_ratio,bp_index
0,0.538462,0.583333,0.285714,0.384615,0.040816,0.051020,0.0,0.0,0.395062,0.443396,...,0.0,0.078261,0.014760,0.008239,0.025075,0.0,1,0.302928,0.383255,0.362408
1,0.769231,0.583333,0.333333,0.487179,0.051020,0.061224,1.0,1.0,0.462963,0.405660,...,0.0,0.086957,0.019373,0.007552,0.035105,1.0,0,0.356838,0.480590,0.484615
2,0.000000,0.666667,0.428571,0.384615,0.030612,0.040816,0.0,0.0,0.290123,0.330189,...,0.0,0.060870,0.019373,0.010299,0.051153,0.0,1,0.417790,0.354270,0.375146
3,0.230769,0.833333,0.619048,0.692308,0.142857,0.112245,0.0,0.0,0.370370,0.452830,...,0.0,0.078261,0.012915,0.008926,0.028084,1.0,0,0.516696,0.568800,0.325218
4,0.153846,0.583333,0.285714,0.378205,0.142857,0.091837,0.0,0.0,0.308642,0.339623,...,0.0,0.060870,0.011993,0.004119,0.015045,0.0,1,0.302928,0.377172,0.386212


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df = scaler.fit_transform(df)

In [92]:
Y = df['smoking']
X = df.drop(['smoking'], axis=1)

Y.head()

0    1
1    0
2    1
3    0
4    1
Name: smoking, dtype: int64

In [93]:
SEED = np.random.seed(44)

X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.25, 
                                                    random_state=SEED, 
                                                    shuffle =True)

print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)


X_train shape is  (148680, 25)
X_test shape is  (49560, 25)
y_train shape is  (148680,)
y_test shape is  (49560,)


In [82]:
xgb_params = {'n_estimators': 727, 
              'max_depth': 44, 
              'min_child_weight': 42.394074475465935, 
              'learning_rate': 0.018945904767046495, 
              'subsample': 0.9976305222111156, 
              'gamma': 0.23054785929528437, 
              'colsample_bytree': 0.4156956766282452, 
              'colsample_bylevel': 0.9225226228188033, 
              'colsample_bynode': 0.686558727709571
             }

# Build the XGBoost model
XGBModel = XGBClassifier(**xgb_params, random_state=SEED)

XGBModel.fit(X_train, y_train)

y_pred = XGBModel.predict(X_test)
y_pred_proba = XGBModel.predict_proba(X_test)[:, 1]

In [70]:
lgbm_params = {'n_estimators': 624, 
               'max_depth': 46, 
               'learning_rate': 0.06953273561619135, 
               'min_child_weight': 2.4187716216112944, 
               'min_child_samples': 230, 
               'subsample': 0.9515130309407626, 
               'subsample_freq': 4, 
               'colsample_bytree': 0.402284262124352, 
               'num_leaves': 71
              }

# Build the LightGBM model
LGBMModel = LGBMClassifier(**lgbm_params, random_state=SEED)
LGBMModel.fit(X_train, y_train)

y_pred = LGBMModel.predict(X_test)
y_pred_proba = LGBMModel.predict_proba(X_test)[:, 1]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 63071, number of negative: 85609
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004366 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2975
[LightGBM] [Info] Number of data points in the train set: 148680, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.424206 -> initscore=-0.305529
[LightGBM] [Info] Start training from score -0.305529


In [59]:
rf = RandomForestClassifier(bootstrap=True, max_depth=40, max_features="sqrt", min_samples_leaf=1, min_samples_split=2, n_estimators=800)



rf.fit(X_train, y_train)
# pred_tr = rf.predict_proba(X_train)
y_pred = rf.predict(X_test)

# pred_train = rf.predict_proba(X_train)
y_pred_proba = rf.predict_proba(X_test)

In [84]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier

# Define base models
base_models = [
    ('rf', RandomForestClassifier(bootstrap=True, max_depth=40, max_features="sqrt", min_samples_leaf=1, min_samples_split=2, n_estimators=800)),
    ('gb', CatBoostClassifier(n_estimators=1000,learning_rate= 0.2,depth= 5))
]

# Define meta-model
meta_model = LogisticRegression()

# Create the stacking model
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Train the stacking model
stacking_model.fit(X_train, y_train)

# Evaluate the model
stacking_accuracy = stacking_model.score(X_test, y_test)
print(f'Stacking Model Accuracy: {stacking_accuracy}')

0:	learn: 0.6111515	total: 69.1ms	remaining: 1m 8s
1:	learn: 0.5629866	total: 77.9ms	remaining: 38.9s
2:	learn: 0.5341611	total: 86.4ms	remaining: 28.7s
3:	learn: 0.5175103	total: 94.7ms	remaining: 23.6s
4:	learn: 0.5080030	total: 103ms	remaining: 20.5s
5:	learn: 0.5009279	total: 112ms	remaining: 18.6s
6:	learn: 0.4939896	total: 122ms	remaining: 17.3s
7:	learn: 0.4891854	total: 129ms	remaining: 16s
8:	learn: 0.4855435	total: 137ms	remaining: 15.1s
9:	learn: 0.4824357	total: 145ms	remaining: 14.3s
10:	learn: 0.4801559	total: 153ms	remaining: 13.7s
11:	learn: 0.4778009	total: 161ms	remaining: 13.2s
12:	learn: 0.4757415	total: 168ms	remaining: 12.8s
13:	learn: 0.4740058	total: 176ms	remaining: 12.4s
14:	learn: 0.4725065	total: 184ms	remaining: 12.1s
15:	learn: 0.4713086	total: 192ms	remaining: 11.8s
16:	learn: 0.4703183	total: 199ms	remaining: 11.5s
17:	learn: 0.4695717	total: 207ms	remaining: 11.3s
18:	learn: 0.4687997	total: 215ms	remaining: 11.1s
19:	learn: 0.4679405	total: 224ms	remai

In [94]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148680 entries, 55242 to 14100
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   age                    148680 non-null  float64
 1   height(cm)             148680 non-null  float64
 2   weight(kg)             148680 non-null  float64
 3   waist(cm)              148680 non-null  float64
 4   eyesight(left)         148680 non-null  float64
 5   eyesight(right)        148680 non-null  float64
 6   hearing(left)          148680 non-null  float64
 7   hearing(right)         148680 non-null  float64
 8   systolic               148680 non-null  float64
 9   relaxation             148680 non-null  float64
 10  fasting blood sugar    148680 non-null  float64
 11  Cholesterol            148680 non-null  float64
 12  triglyceride           148680 non-null  float64
 13  HDL                    148680 non-null  float64
 14  LDL                    148680 non-null

In [96]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Define a new model with Batch Normalization and LeakyReLU activation
from tensorflow.keras import regularizers

model = models.Sequential([
    layers.Dense(256, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(128, kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(64, kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])


model.summary()

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

early_stopping = EarlyStopping(monitor='val_auc', mode='max', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_auc', mode='max', factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=2
)

val_loss, val_accuracy, val_auc = model.evaluate(X_test, y_test, verbose=2)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation AUC: {val_auc:.4f}")



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               6656      
                                                                 
 batch_normalization_2 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization_3 (Batc  (None, 128)              512       
 hNormalization)                                      

In [83]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Assuming SEED, X_train, y_train, X_test, y_test are defined somewhere in your code

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate error rate
error_rate = 1 - accuracy
print(f'Error Rate: {error_rate}')

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1}')

# Calculate precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Calculate recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

from sklearn.metrics import roc_auc_score

# Calculate ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'ROC AUC: {roc_auc}')


Accuracy: 0.7894471347861178
Error Rate: 0.21055286521388217
F1 Score: 0.7624467867143214
Precision: 0.7272332479263474
Recall: 0.801244019138756
ROC AUC: 0.8740878840188717
